<a href="https://colab.research.google.com/github/myl941222/Movie_review_classification/blob/main/MSDS_453_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###############################################################################
### packages required to run code.  Make sure to install all required packages.
###############################################################################
import re,string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
import os
from gensim import corpora
from gensim.models import LsiModel,LdaModel
from gensim import similarities
from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel


from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

import warnings
import random
warnings.filterwarnings('ignore')


In [ ]:
#Custom functions required as part of demonstration
#functions used to process text and create the required TFIDF and Doc2vec Matrices
def clean_doc(doc): 
    #split document into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]         
    # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]
    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

def tfidf(corpus, titles, ngram_range = (1,1)):
    #this is a function to created the tfidf matrix
    Tfidf=TfidfVectorizer(ngram_range= ngram_range)

    #fit the vectorizer using final processed documents.  The vectorizer requires the 
    #stiched back together document.

    TFIDF_matrix=Tfidf.fit_transform(corpus)     

    #creating datafram from TFIDF Matrix
    words = Tfidf.get_feature_names()
    matrix=pd.DataFrame(TFIDF_matrix.toarray(), columns=Tfidf.get_feature_names(), index=titles)
    return matrix


def One_Hot(variable):
    #this is a function to one hot encode the classes
    LE=LabelEncoder()
    LE.fit(variable)
    Label1=LE.transform(variable)
    OHE=OneHotEncoder()
    labels=OHE.fit_transform(Label1.reshape(-1,1)).toarray()
    return labels



In [ ]:
new_stopwords = ['evil','devil','witch','EC_']

In [ ]:
#Section 1: K-Means
def k_means(titles, tfidf_matrix, k=5):
    
    #this is a function to generate the k-means output using the tfidf matrix.  Inputs 
    #to the function include: titles of text, processed text, and desired k value. 
    #Returns dataframe indicating cluster number per document

    km = KMeans(n_clusters=k, random_state =100)
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()

    Dictionary={'Doc Name':titles, 'Cluster':clusters,  'Text': final_processed_text}
    frame=pd.DataFrame(Dictionary, columns=['Cluster', 'Doc Name','Text'])
    #dictionary to store clusters and respective titles
    cluster_title={}

    #note doc2vec clusters will not have individual words due to the vector representation
    #is based on the entire document not indvidual words. As a result, there won't be individual
    #word outputs from each cluster.   
    for i in range(k):
        temp=frame[frame['Cluster']==i]
        temp_title_list=[]
        for title in temp['Doc Name']:
            temp_title_list.append(title)
        cluster_title[i]=temp_title_list

    return cluster_title,clusters,frame



In [ ]:
def plot_tfidf_matrix(cluster_title,clusters,TFIDF_matrix):
    # convert two components as we're plotting points in a two-dimensional plane
    # "precomputed" because we provide a distance matrix
    # we will also specify `random_state` so the plot is reproducible.


    mds = MDS(n_components=4, dissimilarity="precomputed", random_state=1)

    dist = 1 - cosine_similarity(TFIDF_matrix)

    pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

    xs, ys = pos[:, 0], pos[:, 1]


    #set up colors per clusters using a dict.  number of colors must correspond to K
    cluster_colors = {0: 'black', 1: 'grey', 2: 'blue', 3: 'rosybrown', 4: 'firebrick', 
                      5:'red', 6:'darksalmon', 7:'sienna'}


    #set up cluster names using a dict.  
    cluster_dict=cluster_title

    #create data frame that has the result of the MDS plus the cluster numbers and titles
    df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=range(0,len(clusters)))) 

    #group by cluster
    groups = df.groupby('label')

    fig, ax = plt.subplots(figsize=(20,20)) # set size
    ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

    #iterate through groups to layer the plot
    #note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
    for name, group in groups:
        
        r = random.random()
        b = random.random()
        g = random.random()
        color = (r, g, b)
        
        ax.plot(group.x, group.y, marker='o', linestyle='', ms=12,
                label=cluster_dict[name], color=color, 
                mec='none')
        ax.set_aspect('auto')
        ax.tick_params(\
            axis= 'x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelbottom='on')
        ax.tick_params(\
            axis= 'y',         # changes apply to the y-axis
            which='both',      # both major and minor ticks are affected
            left='off',      # ticks along the bottom edge are off
            top='off',         # ticks along the top edge are off
            labelleft='on')

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 30})      #show legend with only 1 point



In [ ]:
#Section 3, 4, 5: Classifiers
def classifiers(x, y, model_type, cv = 3):
    
    #this function is to fit 3 different model scenarios.  Support vector machines, logistic regressions, naive bayes.
    #svm = Support vector machin
    #logistic = Logistic regression
    #naive_bayes = Naive Bayes Multinomial
    
    #can define cv value for cross validation.
    
    #function returns the train test split scores of each model.
    
    if model_type == 'svm':
        print("svm")
        model = SVC()

    elif model_type == 'logistic':
        print("logistic")
        model = LogisticRegression()

    elif model_type == 'naive_bayes':
        print("naive_bayes")
        model = MultinomialNB()
    
    elif model_type == 'randomforest':
        print("randomforest")
        model = RandomForestClassifier()

    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    model.fit(X_train, y_train)

    
    predictions = model.predict(X_test)
    accy = accuracy_score(y_test, predictions)
    
    return accy

In [ ]:
#this section of the code prepares the required variables to run functions set for each 
#section of code

#load data frame and process the text
# os.chdir('/Users/paulhuynh/Documents/School/2021 Summer/Section 56/')

#read in class corpus csv into python
data=pd.read_csv('/content/453_Sec55_Class_Corpus.csv')



In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#adding two columns to the dataframe to store the processed text and tokenized text
data['processed_text'] = data['Text'].apply(lambda x: clean_doc(x))



#creating final processed text variables for matrix creation
final_processed_text = [' '.join(x) for x in data['processed_text'].tolist()]
titles = data['DSI_Title'].tolist()
processed_text = data['processed_text'].tolist()


#generate tfidf for analysis
tfidf_matrix = tfidf(final_processed_text, titles, ngram_range = (1,1))

In [ ]:
#Section 1: Execute K-means clustering
cluster_title,clusters,k_means_df = k_means(titles, tfidf_matrix, k =13)

In [ ]:
#access each cluster by changing the dictionary key value from 0 to k
cluster_title

{0: ['WF_Doc10__Home-Alone_-Reboot',
  'WF_Doc1_Home-Alone-(Review)',
  'WF_Doc2_Heartwarming-Holiday-Family',
  'WF_Doc3_Movie-Review-Home',
  'WF_Doc4_Home-Alone-Film',
  'WF_Doc5_Home-Alone-Still',
  'WF_Doc6_Home-Alone',
  'WF_Doc7__Home-Alone_-Is',
  'WF_Doc8_1990-_Home-Alone_',
  'WF_Doc9__Home-Alone_-Turns'],
 1: ['AJP_Doc10_For_Arrival_Is_A',
  'AJP_Doc1_Denis_Villeneuves_Arrival',
  'AJP_Doc2_Arrival_Arrives_Just',
  'AJP_Doc3_The_Prescient_Wisdom',
  'AJP_Doc4_Arrival_Is_A',
  'AJP_Doc5_Arrival_Is_The',
  'AJP_Doc6_Denis_Villeneuve_Wants',
  'AJP_Doc7_Star_Studded_Arrival',
  'AJP_Doc8_Arrival_The_Film',
  'AJP_Doc9_For_A_Film',
  'CMP_Doc5_Poltergeist',
  'EC_Doc10_Interstellar_Negative',
  'EC_Doc1_Interstellar_Positive',
  'EC_Doc2_Interstellar_Positive',
  'EC_Doc3_Interstellar_Positive',
  'EC_Doc4_Interstellar_Positive',
  'EC_Doc5_Interstellar_Positive',
  'EC_Doc6_Interstellar_Negative',
  'EC_Doc7_Interstellar_Negative',
  'EC_Doc8_Interstellar_Negative',
  'EC_Doc9_

In [ ]:
plot_tfidf_matrix(cluster_title,clusters,tfidf_matrix)

In [ ]:
labels = data['Review Type (pos or neg)'].apply(lambda x: 0 if x.lower().split(' ')[0] == 'negative' else 1)

print(labels)

0      0
1      1
2      1
3      1
4      1
      ..
195    1
196    0
197    0
198    0
199    0
Name: Review Type (pos or neg), Length: 200, dtype: int64


In [ ]:
#Section 2: Support Vector Machines
classifiers(tfidf_matrix, labels, 'svm')

svm


0.4393939393939394

In [ ]:
#Section 3: Support Vector Machines
classifiers(tfidf_matrix, labels, 'logistic')
    


logistic


0.45454545454545453

In [ ]:
#Section 4: Naive Bayes
classifiers(tfidf_matrix, labels, 'naive_bayes')
    


naive_bayes


0.45454545454545453

In [ ]:
#Section 4: Naive Bayes
classifiers(tfidf_matrix, labels, 'randomforest')
    


randomforest


0.45454545454545453

In [ ]:
#Section 1: Latent Semantic Analysis Example
def lsa(tfidf_matrix, terms, n_components = 10):
    #this is a function to execute lsa.  inputs to the function include the tfidf matrix and
    #the desired number of components.
    
    LSA = TruncatedSVD(n_components=10)

    LSA.fit(tfidf_matrix)


    for i, comp in enumerate(LSA.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        print("Topic "+str(i)+": ")
        for t in sorted_terms:
            print(t[0])



In [ ]:
#Section 2: Latent Dirichlet Allocation
def lda(tfidf_matrix, terms, topics = 3, num_words = 10):
    #this is a function to perform lda on the tfidf matrix.  function varibales include:
    #tfidf matrix, desired number of topic, and number of words per topic.

    topics = 3
    num_words = 10
    lda = LatentDirichletAllocation(n_components=topics).fit(tfidf_matrix)

    topic_dict = {}
    for topic_num, topic in enumerate(lda.components_):
        topic_dict[topic_num] = " ".join([terms[i]for i in topic.argsort()[:-num_words - 1:-1]])

    print(topic_dict)

In [ ]:
# #this section of the code prepares the required variables to run functions set for each 
# #section of code

# #read in class corpus csv into python
# data=pd.read_csv(r'/content/453_Sec55_Class_Corpus.csv')


# #adding two columns to the dataframe to store the processed text and tokenized text
# data['processed_text'] = data['Text'].apply(lambda x: clean_doc(x))



# #creating final processed text variables for matrix creation
# final_processed_text = [' '.join(x) for x in data['processed_text'].tolist()]
# titles = data['DSI_Title'].tolist()
# processed_text = data['processed_text'].tolist()


In [ ]:
def plot_lsa(number_of_topics, words):

    ######################################
    #Function to plot the LSA based on user defined numbrer of topics and words
    #EXPERIMENT WITH THESE PARAMETERS topics and words

    #####################################
    model,dictionary,index=create_gensim_lsa_model(processed_text,number_of_topics,words)

    for doc in processed_text:
        vec_bow = dictionary.doc2bow(doc)
        vec_lsi = model[vec_bow]  # convert the query to LSI space
        sims = index[vec_lsi] # perform a similarity query against the corpus

    fig, ax = plt.subplots(figsize=(100, 40))
    cax = ax.matshow(index, interpolation='nearest')
    ax.grid(True)
    plt.xticks(range(len(processed_text)), titles, rotation=90);
    plt.yticks(range(len(processed_text)), titles);
    fig.colorbar(cax)
    plt.show()
    fig.savefig('temp.png', dpi=fig.dpi)
    return model
    

In [ ]:
def plot_lda(number_of_topics, words):

    ######################################
    #Function to plot the LSA based on user defined numbrer of topics and words
    #EXPERIMENT WITH THESE PARAMETERS topics and words

    #####################################
    model2,dictionary2,index2,doctermmatrix2=create_gensim_lda_model(processed_text,number_of_topics,words)

    for doc in processed_text:
        vec_bow2 = dictionary2.doc2bow(doc)
        vec2 = model2[vec_bow2]  # convert the query to embedded space
        sims2 = index2[vec2]  # perform a similarity query against the corpus
        #print(list(enumerate(sims2)))  

    fig, ax = plt.subplots(figsize=(100, 50))
    cax = ax.matshow(index2, interpolation='nearest')
    ax.grid(True)
    plt.xticks(range(len(processed_text)), titles, rotation=90);
    plt.yticks(range(len(processed_text)), titles);
    fig.colorbar(cax)
    plt.show()
    fig.savefig('temp.png', dpi=fig.dpi)


    return model2
    

In [ ]:

#Section 1: Latent Semantic Analysis Using Gensim's implementation
def create_gensim_lsa_model(doc_clean,number_of_topics,words):

    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    index = similarities.MatrixSimilarity(lsamodel[doc_term_matrix])

    return lsamodel,dictionary,index



In [ ]:
#Section 2:  Latent Dirichlet Allocation Using Gensim's implementation
def create_gensim_lda_model(doc_clean,number_of_topics,words):

    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LSA model
    ldamodel = LdaModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)
    # train model
    print(ldamodel.print_topics(num_topics=number_of_topics, num_words=words))
    index = similarities.MatrixSimilarity(ldamodel[doc_term_matrix])



    return ldamodel,dictionary,index,doc_term_matrix



In [ ]:
# #plotting 2 topics and 10 words
# model_2topic_10words = plot_lsa(2, 10)

In [ ]:
# #plotting 4 topics and 10 words
# model_4topic_10words = plot_lsa(4, 10)

In [ ]:
# #plotting 10 topics and 10 words
# model_10topic_10words =plot_lsa(10, 10)

In [ ]:
# #plotting 20 topics and 10 words
# model_20topic_10words =plot_lsa(20, 10)

In [ ]:
#compare LSA model coherence

topics = [2,4,10,20]
coherence_values = []
for t in topics:
    lsamodel_1,dictionary_1,index_1 = create_gensim_lsa_model(processed_text,t,30)


    coherence_model_lsa = CoherenceModel(model=lsamodel_1, dictionary=dictionary_1, texts=processed_text, coherence='c_v')
    coherence_lsa = coherence_model_lsa.get_coherence()
    coherence_values.append(coherence_lsa)

coherence ={'2 topic 30 words':coherence_values[0],
           '4 topic 30 words': coherence_values[1],
          '10 topic 30 words': coherence_values[1],
           '20 topic 30 words':coherence_values[2],
            }   

print(coherence)

[(0, '0.358*"movie" + 0.172*"first" + 0.168*"movies" + 0.160*"story" + 0.119*"films" + 0.117*"world" + 0.116*"action" + 0.112*"never" + 0.110*"little" + 0.108*"character" + 0.107*"family" + 0.102*"though" + 0.101*"could" + 0.100*"would" + 0.098*"something" + 0.096*"another" + 0.094*"horror" + 0.089*"conjuring" + 0.088*"really" + 0.087*"title" + 0.083*"still" + 0.080*"every" + 0.079*"people" + 0.079*"around" + 0.078*"characters" + 0.077*"director" + 0.077*"makes" + 0.076*"might" + 0.075*"things" + 0.073*"almost"'), (1, '0.434*"conjuring" + 0.257*"devil" + 0.255*"warrens" + 0.232*"movie" + 0.193*"horror" + -0.179*"casino" + -0.175*"royale" + -0.126*"craig" + -0.125*"action" + 0.121*"movies" + -0.111*"world" + 0.105*"lorraine" + 0.093*"family" + 0.093*"chaves" + -0.092*"first" + -0.089*"bonds" + 0.087*"johnson" + 0.086*"farmiga" + 0.083*"warren" + 0.079*"house" + -0.076*"martian" + -0.076*"mission" + -0.073*"watney" + 0.073*"wilson" + -0.072*"review" + -0.070*"space" + -0.069*"reeves" + 0

In [ ]:
# model_2topic_10words =plot_lda(2, 10)

In [ ]:
# model_4topic_10words =plot_lda(4, 10)

In [ ]:
# model_10topic_10words =plot_lda(10, 10)

In [ ]:
# model_20topic_10words =plot_lda(20, 10)

In [ ]:
#compare LDA model coherence

topics = [2, 4, 10,20]
coherence_values = []
for t in topics:
    ldamodel,dictionary,index, matrix = create_gensim_lda_model(processed_text,t,10)


    coherence_model_lda = CoherenceModel(model=ldamodel, dictionary=dictionary, texts=processed_text, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

    
coherence ={'5 topic 10 words':coherence_values[0],
           '4 topic 10 words': coherence_values[1],
           '10 topic 10 words':coherence_values[2],
            '20 topic 10 words':coherence_values[3],
            }   

print(coherence)

[(0, '0.006*"movie" + 0.003*"first" + 0.003*"action" + 0.003*"movies" + 0.003*"story" + 0.002*"world" + 0.002*"films" + 0.002*"character" + 0.002*"little" + 0.002*"something"'), (1, '0.006*"movie" + 0.003*"story" + 0.003*"first" + 0.002*"family" + 0.002*"movies" + 0.002*"never" + 0.002*"little" + 0.002*"films" + 0.002*"director" + 0.002*"though"')]
[(0, '0.006*"movie" + 0.003*"story" + 0.003*"first" + 0.003*"action" + 0.002*"films" + 0.002*"little" + 0.002*"movies" + 0.002*"world" + 0.002*"family" + 0.002*"could"'), (1, '0.005*"movie" + 0.003*"action" + 0.003*"first" + 0.003*"world" + 0.002*"never" + 0.002*"story" + 0.002*"movies" + 0.002*"title" + 0.002*"films" + 0.002*"though"'), (2, '0.009*"movie" + 0.004*"movies" + 0.004*"first" + 0.003*"conjuring" + 0.003*"story" + 0.002*"would" + 0.002*"character" + 0.002*"little" + 0.002*"something" + 0.002*"action"'), (3, '0.004*"movie" + 0.003*"story" + 0.003*"family" + 0.003*"movies" + 0.002*"never" + 0.002*"horror" + 0.002*"first" + 0.002*"l